# Bayesian model selection and linear regression

This notebook uses [Bayesian selection for linear regression with basis functions](https://arxiv.org/abs/1512.04823) in order to (partially) answer [question #2231975](https://math.stackexchange.com/questions/2231975/interpolating-a-given-data-by-a-simple-function/2232010#2232010) in Math StackExchange.

The necessary code can be found in [bitbucket](https://bitbucket.org/mdbenito/modelselection).

In [ ]:
import sys
sys.path.append("../src/")
from Hypotheses import *
from ModelSelection import LinearRegression
from Plots import updateMAPFitPlot, updateProbabilitiesPlot
import numpy as np
from sklearn import preprocessing
import matplotlib.pyplot as pl
%matplotlib notebook

After importing the necessary modules, we load data and normalize it to have zero mean and variance 1. This is required to avoid numerical issues: for large values of the target values, some probabilities in the computations become zero because of the exponential function ($e^{-t}$ becomes almost zero for relatively small values of $t$).

In [ ]:
data = np.loadtxt('data-2231875.txt', delimiter=',', skiprows=1)
data[:,1] = preprocessing.scale(data[:,1])
pl.title("The (normalized) dataset")
_ = pl.plot(data[:,0], data[:,1])
#pl.savefig('data.svg')

We now prepare some set of hypothesis spaces to be tested against each other. Because it's easy and alrady implemented in the repo, we take two polynomial and two trigonometric families of basis functions.

In [ ]:
sigma = data[:, 1].std()  # observation noise sigma (should be approx. 1 after scaling)
hc = HypothesisCollection()
hc.append(PolynomialHypothesis(M=6, variance=2, noiseVariance=sigma**2))
hc.append(PolynomialHypothesis(M=8, variance=2, noiseVariance=sigma**2))
hc.append(TrigonometricHypothesis(halfM=4, variance=2, noiseVariance=sigma**2))
hc.append(TrigonometricHypothesis(halfM=6, variance=2, noiseVariance=sigma**2))

lr = LinearRegression(hc, sigma)

We now perform bayesian updates to our belief in each hypothesis space. Each data point is fed to the `LinearRegression` object which then performs:
1. Estimation of the weights for each hypothesis.
2. Computation of the posterior probability of each hypothesis, given the data.

In [ ]:
%%time
ymin, ymax = min(data[:,1]), max(data[:,1])
# Looping is ugly, but it is what it is! :P
for x, y in data:
    lr.update(x, y)

# MAP values for the weights w_j
wmap = [param.mean for param in lr.parameter]

In [ ]:
fig, (ax1, ax2) = pl.subplots(2)
updateMAPFitPlot(ax1, lr.XHist, lr.hypotheses, wmap, 0.005) 
ax1.plot(lr.XHist, lr.THist, 'k+', ms=4, alpha=0.5)  # plot the data points
ax1.set_title("Data and MAP fits")
updateProbabilitiesPlot(ax2, lr)
ax2.set_title("Incremental model probability")
fig.subplots_adjust(hspace=0.5)
#pl.savefig('mapfits.svg')
_ = pl.show()

The winner among the hypotheses proposed is clearly the Trigonometric hypothesis  ($H_2$) with $M=8$ basis functions:

$$\phi_j (x) = \cos (\pi jx)\  \text{ for }\ j = 0, \ldots, M - 1,$$

so that our best candidate is

$$f(x) = \sum_{j=0}^8 w_j \phi_j (x). $$

The specific values of the weights $w_j$ are taken from from the _a posteriori_ distribution computed (Gaussian since we started with a Gaussian prior). Their MAP values are:

In [ ]:
wmap[2].round(2).flatten()

Note how the model comparison rejects the seemingly better `Trig11` hypothesis at around half the dataset, and leans in favor of `Trig7` which looks like a poorer fit. This is because model complexity is penalized. `Trig11` is a wildly oscillating polynomial beyond the interval considered whereas `Trig7` is a bit more tame.

In [ ]:
xx = np.linspace(-1,2,200)
for h, w, l in zip(lr.hypotheses[2:], wmap[2:], ['Trig7', 'Trig11']):
    pl.plot(xx, [np.dot(h.evaluate(x).flatten(), w) for x in xx], label=l)
pl.title("Complexity in competing hypotheses")
_ = pl.legend()
#pl.savefig('complexity.svg')

This doesn't mean however that either is a good fit (nor that extrapolating beyond the range of the dataset would be wise, no matter how good the fit is), only that one is better than the other. At this point we would need to try more hypothesis spaces, perhaps ones with functions with compact support at multiple scales and locations.